In [1]:
import yelp
import requests
import argparse
import json
import pprint
import sys
import urllib
import urllib2

import oauth2


In [2]:
yelp_api = yelp.Api(consumer_key = '9ZYLXFQyKrwt5o5q0oK26A',
                    consumer_secret = 'xILWnEKR1oV5kZqhuw7r-9YD2BQ',
                    access_token_key = 'rdSNVzPsap3MSQwWUZ0IapNSI7X7q-8K',
                    access_token_secret = 'G5aTHtgx6j8ZWLfbRujl5ZwFSnM')

In [3]:
API_HOST = 'api.yelp.com'
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3
SEARCH_PATH = '/v2/search/'
BUSINESS_PATH = '/v2/business/'


In [4]:
# -*- coding: utf-8 -*-
"""
Yelp API v2.0 code sample.
This program demonstrates the capability of the Yelp API version 2.0
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.
Please refer to http://www.yelp.com/developers/documentation for the API documentation.
This program requires the Python oauth2 library, which you can install via:
`pip install -r requirements.txt`.
Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
import argparse
import json
import pprint
import sys
import urllib
import urllib2

import oauth2


API_HOST = 'api.yelp.com'
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3
SEARCH_PATH = '/v2/search/'
BUSINESS_PATH = '/v2/business/'

# OAuth credential placeholders that must be filled in by users.
CONSUMER_KEY = '9ZYLXFQyKrwt5o5q0oK26A'
CONSUMER_SECRET = 'xILWnEKR1oV5kZqhuw7r-9YD2BQ'
TOKEN = 'rdSNVzPsap3MSQwWUZ0IapNSI7X7q-8K'
TOKEN_SECRET = 'G5aTHtgx6j8ZWLfbRujl5ZwFSnM'


def request(host, path, url_params=None):
    """Prepares OAuth authentication and sends the request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        urllib2.HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = 'https://{0}{1}?'.format(host, urllib.quote(path.encode('utf8')))

    consumer = oauth2.Consumer(CONSUMER_KEY, CONSUMER_SECRET)
    oauth_request = oauth2.Request(method="GET", url=url, parameters=url_params)

    oauth_request.update(
        {
            'oauth_nonce': oauth2.generate_nonce(),
            'oauth_timestamp': oauth2.generate_timestamp(),
            'oauth_token': TOKEN,
            'oauth_consumer_key': CONSUMER_KEY
        }
    )
    token = oauth2.Token(TOKEN, TOKEN_SECRET)
    oauth_request.sign_request(oauth2.SignatureMethod_HMAC_SHA1(), consumer, token)
    signed_url = oauth_request.to_url()
    
    print u'Querying {0} ...'.format(url)

    conn = urllib2.urlopen(signed_url, None)
    try:
        response = json.loads(conn.read())
    finally:
        conn.close()

    return response

def search(term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """
    
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, url_params=url_params)

def get_business(business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path)

def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(term, location)

    businesses = response.get('businesses')

    if not businesses:
        print u'No businesses for {0} in {1} found.'.format(term, location)
        return

    business_id = businesses[0]['id']

    print u'{0} businesses found, querying business info for the top result "{1}" ...'.format(
        len(businesses),
        business_id
    )

    response = get_business(business_id)

    print u'Result for business "{0}" found:'.format(business_id)
    pprint.pprint(response, indent=2)


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM, type=str, help='Search term (default: %(default)s)')
    parser.add_argument('-l', '--location', dest='location', default=DEFAULT_LOCATION, type=str, help='Search location (default: %(default)s)')

    input_values = parser.parse_args()

    try:
        query_api(input_values.term, input_values.location)
    except urllib2.HTTPError as error:
        sys.exit('Encountered HTTP error {0}. Abort program.'.format(error.code))


if __name__ == '__main__':
    main()

usage: __main__.py [-h] [-q TERM] [-l LOCATION]
__main__.py: error: unrecognized arguments: -f /home/mikez/.ipython/profile_default/security/kernel-4e1b6d50-c978-4aa4-a408-c5bbd92ef1f1.json --profile-dir /home/mikez/.ipython/profile_default


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [5]:
def get_business_ids(term, location, radius, maxlimit=20):
    offset = 0
    businesses = []
    resume = True
    while resume:
        url_params = {
            'term': term.replace(' ', '+'),
            'location': location.replace(' ', '+'),
            'limit': maxlimit,
            'radius_filter': radius,
            'offset': offset
        }
        output = request(API_HOST, SEARCH_PATH, url_params = url_params)
        n_records = len(output['businesses'])
        print n_records
        if n_records < 20:
            resume = False
        for i in range(n_records):
            businesses.append(output['businesses'][i]['id']) 
        offset += n_records
    return businesses

In [6]:
small_bizlist = get_business_ids('Restaurants', 'San Francisco, CA', 500)

Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
19


In [16]:
big_bizlist = get_business_ids('Restaurant', 'San Francisco, CA', 1880)

Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api.yelp.com/v2/search/? ...
20
Querying https://api

In [17]:
print len(big_bizlist)
#small_bizlist

999


In [26]:
small_bizlist

[u'nani-san-francisco',
 u'little-griddle-san-francisco',
 u'alta-ca-san-francisco',
 u'zuni-caf\xe9-san-francisco-3',
 u'big-chef-toms-belly-burgers-san-francisco',
 u'grandmas-deli-and-cafe-san-francisco',
 u'cala-san-francisco-2',
 u'minas-brazilian-restaurant-and-cacha\xe7aria-san-francisco',
 u'pl\xe4j-scandinavian-restaurant-and-bar-san-francisco',
 u'bon-marche-brasserie-and-bar-san-francisco',
 u'1601-bar-and-kitchen-san-francisco',
 u'teds-market-and-delicatessen-san-francisco',
 u'jardini\xe8re-san-francisco-2',
 u'taqueria-cazadores-san-francisco-2',
 u'ananda-fuara-san-francisco',
 u'20th-century-cafe-san-francisco-2',
 u'dirty-water-san-francisco-4',
 u'the-grove-hayes-san-francisco',
 u'rons-pit-stop-san-francisco',
 u'delessio-market-and-bakery-san-francisco',
 u'nojo-san-francisco',
 u'gaslamp-cafe-san-francisco',
 u'the-chairman-truck-san-francisco',
 u'the-rickshaw-grill-san-francisco-2',
 u'lers-ros-thai-san-francisco-2',
 u'eden-cafe-san-francisco',
 u'shido-san-fra

In [88]:
for b in small_bizlist:
    print b

nani-san-francisco
little-griddle-san-francisco
alta-ca-san-francisco
zuni-café-san-francisco-3
big-chef-toms-belly-burgers-san-francisco
grandmas-deli-and-cafe-san-francisco
cala-san-francisco-2
minas-brazilian-restaurant-and-cachaçaria-san-francisco
pläj-scandinavian-restaurant-and-bar-san-francisco
bon-marche-brasserie-and-bar-san-francisco
1601-bar-and-kitchen-san-francisco
teds-market-and-delicatessen-san-francisco
jardinière-san-francisco-2
taqueria-cazadores-san-francisco-2
ananda-fuara-san-francisco
20th-century-cafe-san-francisco-2
dirty-water-san-francisco-4
the-grove-hayes-san-francisco
rons-pit-stop-san-francisco
delessio-market-and-bakery-san-francisco
nojo-san-francisco
gaslamp-cafe-san-francisco
the-chairman-truck-san-francisco
the-rickshaw-grill-san-francisco-2
lers-ros-thai-san-francisco-2
eden-cafe-san-francisco
shido-san-francisco
the-boneyard-san-francisco
pause-wine-bar-san-francisco
zaoh-restaurant-san-francisco
grilled-cheez-guy-san-francisco
ninas-cafe-san-franc

In [81]:
tiny_bizlist

SyntaxError: invalid syntax (<ipython-input-81-9f43b3468817>, line 1)

In [21]:
# Imports
import string
from lxml import html
import requests

# Yelp unique url endings for each restaurant
RESTAURANTS = biz99[0:5]

start_urls = [ 'http://www.yelp.com/biz/%s' % s for s in RESTAURANTS]
num_reviews = 2000 # Number of reviews you want
page_order = range(0, (num_reviews+1), 40)
review_dict = {}

for ur in start_urls:
    for o in page_order:
        page = requests.get(ur + ("?start=%s" % o))
        tree = html.fromstring(page.text)

        # This will extract the name of the business
        title = tree.xpath('//h1[@itemprop="name"]/text()')
        title = title[0].strip()

        # This will create a list of reviews
        reviews = tree.xpath('//p[@itemprop="description"]/text()')
        mod_reviews = []
        for rev in reviews:
            mod_rev = filter(lambda x: x in string.printable, rev)
            mod_reviews.append(mod_rev)


        # This will add the reviews to a dictionary
        if title in review_dict:
            review_dict[title] += mod_reviews
        else:
            review_dict[title] = mod_reviews

In [18]:
biz99 = ["nani-san-francisco",
"little-griddle-san-francisco",
"alta-ca-san-francisco",
"zuni-café-san-francisco-3",
"big-chef-toms-belly-burgers-san-francisco",
"grandmas-deli-and-cafe-san-francisco",
"cala-san-francisco-2",
"minas-brazilian-restaurant-and-cachaçaria-san-francisco",
"pläj-scandinavian-restaurant-and-bar-san-francisco",
"bon-marche-brasserie-and-bar-san-francisco",
"1601-bar-and-kitchen-san-francisco",
"teds-market-and-delicatessen-san-francisco",
"jardinière-san-francisco-2",
"taqueria-cazadores-san-francisco-2",
"ananda-fuara-san-francisco",
"20th-century-cafe-san-francisco-2",
"dirty-water-san-francisco-4",
"the-grove-hayes-san-francisco",
"rons-pit-stop-san-francisco",
"delessio-market-and-bakery-san-francisco",
"nojo-san-francisco",
"gaslamp-cafe-san-francisco",
"the-chairman-truck-san-francisco",
"the-rickshaw-grill-san-francisco-2",
"lers-ros-thai-san-francisco-2",
"eden-cafe-san-francisco",
"shido-san-francisco",
"the-boneyard-san-francisco",
"pause-wine-bar-san-francisco",
"zaoh-restaurant-san-francisco",
"grilled-cheez-guy-san-francisco",
"ninas-cafe-san-francisco-2",
"the-sage-cafe-san-francisco",
"paellas-ole-san-francisco",
"the-market-san-francisco",
"all-star-cafe-san-francisco",
"the-grove-san-francisco",
"hayes-street-grill-san-francisco",
"espetus-churrascaria-san-francisco",
"mini-cafe-san-francisco",
"brass-knuckle-san-francisco",
"azalinas-san-francisco-2",
"the-pastry-cupboard-san-francisco-3",
"lunchbox-cafe-san-francisco",
"south-at-sfjazz-san-francisco",
"oui-chef-san-francisco",
"wes-burger-san-francisco",
"eat-curbside-san-francisco",
"café-at-the-opera-at-the-sfwm-san-francisco",
"los-compadres-taco-truck-san-francisco",
"don-agapitos-san-francisco",
"volare-pizza-and-restaurant-san-francisco",
"go-getters-pizza-san-francisco-3",
"shanghai-china-restaurant-san-francisco",
"lacys-malaysian-crepes-san-francisco",
"bayou-by-the-bay-san-francisco",
"creperie-saint-germain-san-francisco-3",
"pierogisf-san-francisco",
"gracies-cafe-san-francisco",
"little-red-riding-truck-san-francisco",
"we-sushi-san-francisco",
"red-rooster-tacos-san-francisco",
"ultimate-souvlaki-san-francisco-2",
"tower-cafe-san-francisco-2",
"allstar-donuts-and-sandwich-san-francisco",
"subway-san-francisco-51",
"east-of-eden-san-francisco",
"martin-and-hunter-foods-san-francisco",
"crepes-escape-san-francisco",
"adams-grub-truck-san-francisco-2",
"hôtel-biron-san-francisco-2",
"new-home-menu-restaurant-san-francisco",
"horobitsu-no-ie-san-francisco",
"san-francisco-pizza-company-san-francisco",
"tacomyface-san-francisco",
"broth-baby-san-francisco",
"heaven-no-title",
"sfq-original-sf-bbq-sauce-san-francisco",
"kim-wong-keltner-san-francisco",
"chili-dogs-san-francisco",
"sky-high-cafe-san-francisco",
"orlandos-san-francisco",
"girl-friday-zeppole-san-francisco",
"sugar-lounge-san-francisco",
"chicago-style-hot-dogs-san-francisco",
"archimedes-kitchen-san-francisco",
"soup-lady-at-market-and-van-ness-san-francisco",
"native-sons-bbq-san-francisco",
"uma-casa-san-francisco",
"sfo-mobile-cantina-san-francisco",
"tamales-la-oaxaquena-san-francisco",
"youki-ramen-san-francisco",
"churro-vendor-san-francisco",
"anthony-and-alices-bistro-san-francisco",
"bank-of-america-cafeteria-san-francisco",
"fins-on-the-hoof-san-francisco",
"desperation-bakehouse-san-francisco",
"demi-pêche-san-francisco-2",
"new-york-best-buffalo-wings-san-francisco"]

'zuni-caf\xc3\xa9-san-francisco-3'

In [23]:
len(review_dict)

5

In [24]:
review_dict

KeyError: 0

In [31]:
for i in big_bizlist:
    print '"'+ i+'"'+','

"brendas-french-soul-food-san-francisco",
"rickybobby-san-francisco",
"the-chairman-san-francisco-2",
"so-san-francisco-4",
"the-flying-falafel-san-francisco-3",
"hogwash-san-francisco",
"nostra-san-francisco",
"chez-maman-west-san-francisco-2",
"counter-offer-san-francisco",
"burma-love-san-francisco-2",
"nani-san-francisco",
"straw-san-francisco",
"rustys-southern-san-francisco",
"pianofight-san-francisco",
"rove-kitchen-san-francisco",
"the-crafty-fox-ale-house-san-francisco",
"kozs-kitchen-san-francisco",
"citizens-band-san-francisco",
"suppenküche-san-francisco-2",
"saigon-sandwich-san-francisco",
"tommys-joynt-san-francisco",
"elmira-rosticceria-san-francisco",
"azucar-lounge-san-francisco-4",
"montesacro-pinseria-romana-enoteca-san-francisco-4",
"souvla-san-francisco",
"little-griddle-san-francisco",
"elephant-sushi-golden-gate-san-francisco-6",
"chow-san-francisco-3",
"ikes-place-san-francisco",
"alta-ca-san-francisco",
"no-no-burger-san-francisco",
"bite-san-francisco",
"kitch

In [32]:
bigbizlist = ["brendas-french-soul-food-san-francisco",
"rickybobby-san-francisco",
"the-chairman-san-francisco-2",
"so-san-francisco-4",
"the-flying-falafel-san-francisco-3",
"hogwash-san-francisco",
"nostra-san-francisco",
"chez-maman-west-san-francisco-2",
"counter-offer-san-francisco",
"burma-love-san-francisco-2",
"nani-san-francisco",
"straw-san-francisco",
"rustys-southern-san-francisco",
"pianofight-san-francisco",
"rove-kitchen-san-francisco",
"the-crafty-fox-ale-house-san-francisco",
"kozs-kitchen-san-francisco",
"citizens-band-san-francisco",
"suppenküche-san-francisco-2",
"saigon-sandwich-san-francisco",
"tommys-joynt-san-francisco",
"elmira-rosticceria-san-francisco",
"azucar-lounge-san-francisco-4",
"montesacro-pinseria-romana-enoteca-san-francisco-4",
"souvla-san-francisco",
"little-griddle-san-francisco",
"elephant-sushi-golden-gate-san-francisco-6",
"chow-san-francisco-3",
"ikes-place-san-francisco",
"alta-ca-san-francisco",
"no-no-burger-san-francisco",
"bite-san-francisco",
"kitchen-story-san-francisco",
"hopwater-distribution-san-francisco",
"dragoneats-san-francisco",
"mission-beach-cafe-san-francisco-4",
"zuni-café-san-francisco-3",
"the-willows-san-francisco",
"rich-table-san-francisco",
"catheads-bbq-san-francisco",
"big-chef-toms-belly-burgers-san-francisco",
"octavia-san-francisco",
"crepes-a-go-go-san-francisco-2",
"dotties-true-blue-cafe-san-francisco",
"rosamunde-sausage-grill-san-francisco",
"farm-table-san-francisco",
"lunchpad-san-francisco",
"the-monks-kettle-san-francisco",
"mikkeller-bar-san-francisco",
"woodhouse-fish-company-san-francisco",
"grandmas-deli-and-cafe-san-francisco",
"monsieur-benjamin-san-francisco",
"izakaya-roku-san-francisco",
"de-afghanan-kabob-house-san-francisco-2",
"nara-restaurant-and-sake-bar-san-francisco",
"mazzat-san-francisco",
"mission-public-san-francisco",
"kama-sushi-san-francisco-10",
"cala-san-francisco-2",
"basil-thai-restaurant-and-bar-san-francisco",
"two-sisters-bar-and-books-san-francisco",
"minas-brazilian-restaurant-and-cachaçaria-san-francisco",
"absinthe-brasserie-and-bar-san-francisco-2",
"jin-mi-korean-cuisine-san-francisco",
"pläj-scandinavian-restaurant-and-bar-san-francisco",
"shizen-vegan-sushi-bar-and-izakaya-san-francisco",
"soul-food-city-san-francisco-2",
"domo-san-francisco",
"upcider-san-francisco",
"flying-pig-bistro-pub-san-francisco-2",
"bon-marche-brasserie-and-bar-san-francisco",
"liholiho-yacht-club-san-francisco-2",
"redford-san-francisco-2",
"walzwerk-san-francisco",
"1601-bar-and-kitchen-san-francisco",
"el-castillito-san-francisco-2",
"chambers-san-francisco-3",
"tú-lan-san-francisco-4",
"papito-hayes-san-francisco",
"poc-chuc-san-francisco",
"miss-saigon-san-francisco-2",
"sweet-woodruff-san-francisco",
"kokio-republic-san-francisco",
"kappa-japanese-restaurant-san-francisco",
"butter-san-francisco",
"soul-groove-san-francisco",
"the-steamin-burger-san-francisco",
"basil-canteen-san-francisco",
"pica-pica-arepa-kitchen-san-francisco",
"petit-crenn-san-francisco",
"teds-market-and-delicatessen-san-francisco",
"powered-by-pork-san-francisco",
"sunrise-cafe-san-francisco-2",
"moms-bun-mi-san-francisco",
"jardinière-san-francisco-2",
"izakaya-rintaro-san-francisco",
"ssisso-san-francisco",
"nob-hill-grille-san-francisco",
"hai-ky-mi-gia-san-francisco",
"taqueria-cazadores-san-francisco-2",
"the-hall-san-francisco",
"lers-ros-thai-san-francisco",
"libby-jane-cafe-san-francisco-2",
"red-door-cafe-san-francisco-3",
"aria-korean-american-snack-bar-san-francisco",
"biergarten-san-francisco",
"yamo-san-francisco",
"ananda-fuara-san-francisco",
"moya-san-francisco",
"the-sycamore-san-francisco",
"el-capitan-san-francisco",
"izakaya-house-san-francisco",
"stone-bowl-plus-san-francisco",
"mortys-delicatessen-san-francisco",
"ryokos-japanese-restaurant-and-bar-san-francisco",
"boxing-room-san-francisco",
"tony-baloneys-san-francisco",
"pampalasa-san-francisco",
"o-toro-sushi-san-francisco",
"truly-mediterranean-san-francisco",
"fino-ristorante-and-bar-san-francisco",
"ak-subs-san-francisco",
"manoras-thai-cuisine-san-francisco",
"farmhouse-kitchen-thai-cuisine-san-francisco",
"mozzeria-san-francisco",
"chantal-guillon-san-francisco",
"v-cafe-san-francisco",
"taqueria-los-coyotes-san-francisco",
"kin-khao-san-francisco-2",
"the-sandwich-place-san-francisco",
"20th-century-cafe-san-francisco-2",
"tadu-ethiopian-kitchen-san-francisco-3",
"dirty-water-san-francisco-4",
"arlequin-cafe-and-food-to-go-san-francisco",
"universal-cafe-san-francisco-5",
"chabaa-thai-cuisine-san-francisco-2",
"hog-and-rocks-san-francisco",
"wings-plus-san-francisco",
"rye-project-san-francisco",
"mercury-cafe-san-francisco",
"lil-burma-san-francisco",
"four-seasons-restaurant-san-francisco-2",
"the-lab-by-napaology-san-francisco-3",
"cafe-zazo-san-francisco",
"mau-san-francisco",
"fly-bar-and-restaurant-san-francisco",
"cholo-soy-san-francisco",
"bar-tartine-san-francisco",
"dirty-water-san-francisco-4",
"bodega-bistro-san-francisco-2",
"lazy-bear-san-francisco-5",
"farmerbrown-san-francisco-2",
"rice-paper-scissors-san-francisco-2",
"hyde-away-blues-bbq-san-francisco",
"pakwan-restaurant-16th-street-san-francisco",
"doña-mago-san-francisco-3",
"asiasf-san-francisco",
"chai-bar-by-david-rio-san-francisco",
"pearls-deluxe-burgers-san-francisco-3",
"delfina-san-francisco",
"show-dogs-san-francisco",
"orenchi-beyond-san-francisco-6",
"bergerac-san-francisco",
"pancho-villa-taqueria-san-francisco",
"aatxe-san-francisco-2",
"a-la-turca-restaurant-san-francisco",
"pacific-plaza-cafe-san-francisco",
"un-cafecito-san-francisco",
"craftsman-and-wolves-san-francisco",
"pho-tan-hoa-san-francisco",
"sauce-san-francisco",
"belly-good-cafe-and-crepes-san-francisco",
"tycoon-thai-restaurant-san-francisco",
"house-of-thai-2-san-francisco",
"cafe-zitouna-san-francisco",
"gallardos-mexican-restaurant-san-francisco",
"the-grove-hayes-san-francisco",
"le-cafe-du-soleil-san-francisco-3",
"cadillac-bar-and-grill-san-francisco",
"pho-2000-san-francisco",
"pizzeria-delfina-san-francisco",
"tacolicious-san-francisco-7",
"cease-and-desist-san-francisco",
"707-sutter-san-francisco",
"pharaohs-mediterranean-sandwich-san-francisco",
"proposition-chicken-san-francisco",
"west-of-pecos-san-francisco",
"rons-pit-stop-san-francisco",
"olivos-san-francisco",
"nickies-san-francisco",
"wonderland-restaurant-san-francisco",
"white-horse-tavern-and-restaurant-san-francisco",
"mangos-cafe-san-francisco",
"wok-shop-cafe-san-francisco",
"delessio-market-and-bakery-san-francisco",
"taqueria-castillo-mason-san-francisco",
"gestalt-haus-san-francisco",
"huxley-san-francisco",
"locanda-san-francisco",
"mazza-luna-san-francisco",
"dolce-amore-san-francisco",
"waraku-san-francisco-2",
"taqueria-cancún-san-francisco-5",
"jasmine-garden-san-francisco",
"aicha-san-francisco",
"la-fina-estampa-san-francisco",
"odang-udon-san-francisco-3",
"commonwealth-san-francisco",
"nojo-san-francisco",
"wayo-sushi-san-francisco",
"dear-mom-san-francisco",
"los-yaquis-san-francisco",
"turtle-tower-san-francisco-6",
"taylor-street-coffee-shop-san-francisco",
"aq-restaurant-and-bar-san-francisco",
"una-pizza-napoletana-san-francisco",
"the-chairman-truck-san-francisco",
"mint-cafe-san-francisco",
"golden-era-vegan-restaurant-san-francisco-2",
"gaslamp-cafe-san-francisco",
"moulin-rouge-restaurant-san-francisco",
"patxis-pizza-san-francisco-2",
"l-acajou-bakery-and-cafe-san-francisco",
"la-oaxaqueña-san-francisco-2",
"paulines-pizza-and-wine-bar-san-francisco-3",
"destino-san-francisco-3",
"spinnerie-san-francisco",
"dobbs-ferry-restaurant-san-francisco",
"eleven-o-one-san-francisco",
"daniels-cafe-san-francisco-2",
"sammys-cafe-san-francisco",
"mong-thu-cafe-san-francisco",
"yamasho-san-francisco-2",
"sushi-zone-san-francisco",
"perilla-san-francisco-4",
"nihon-whisky-lounge-san-francisco",
"little-star-pizza-san-francisco-2",
"honey-bistro-san-francisco",
"shalimar-restaurant-san-francisco",
"teeth-sf-san-francisco",
"emanuel-coffee-san-francisco-4",
"bar-agricole-san-francisco",
"clay-oven-on-haight-san-francisco",
"colibri-mexican-bistro-san-francisco",
"54-mint-ristorante-italiano-san-francisco-2",
"o3-bistro-and-lounge-san-francisco-3",
"sanraku-san-francisco-2",
"macha-cafe-san-francisco",
"balompie-cafe-san-francisco",
"mela-tandoori-kitchen-san-francisco-2",
"house-of-thai-san-francisco-2",
"zeitgeist-san-francisco",
"homeskillet-san-francisco",
"aveline-san-francisco",
"balo-vietnamese-eatery-san-francisco",
"mission-street-oyster-bar-san-francisco",
"los-hermanos-taqueria-san-francisco",
"la-boulange-de-hayes-san-francisco-2",
"el-castillito-taqueria-san-francisco-2",
"lahore-karahi-san-francisco-2",
"roccos-café-san-francisco-3",
"wing-wings-san-francisco",
"osha-thai-restaurant-san-francisco-7",
"kui-shin-bo-san-francisco-2",
"taqueria-castillo-b-san-francisco",
"mission-cheese-san-francisco",
"sutter-st-cafe-san-francisco",
"mission-banh-mi-san-francisco",
"oro-san-francisco",
"dusty-buns-san-francisco-9",
"mission-bowling-club-san-francisco",
"garage-cafe-san-francisco",
"proposition-chicken-food-truck-san-francisco",
"last-drop-tavern-san-francisco",
"chutney-san-francisco",
"yemenis-restaurant-san-francisco-7",
"the-rickshaw-grill-san-francisco-2",
"stacks-san-francisco",
"ramen-izakaya-goku-san-francisco",
"sams-diner-san-francisco",
"new-star-market-san-francisco",
"giordano-bros-san-francisco-2",
"namu-gaji-san-francisco",
"crepes-ooh-la-la-san-francisco-2",
"maruya-san-francisco",
"jt-restaurant-san-francisco",
"lers-ros-thai-san-francisco-2",
"love-n-haight-deli-san-francisco",
"bitters-bock-and-rye-san-francisco",
"cybelles-pizza-and-ice-cream-san-francisco-2",
"red-chilli-san-francisco",
"split-pea-seduction-san-francisco",
"gracias-madre-san-francisco",
"grubstake-san-francisco",
"anh-hong-restaurant-san-francisco",
"yuzuki-japanese-eatery-san-francisco",
"thorough-bread-and-pastry-san-francisco",
"little-delhi-san-francisco",
"double-decker-san-francisco",
"eden-cafe-san-francisco",
"the-european-san-francisco",
"iza-ramen-san-francisco-2",
"salumeria-san-francisco-2",
"red-crawfish-san-francisco",
"urban-fish-san-francisco-2",
"tuyet-mai-san-francisco",
"an-japanese-restaurant-san-francisco-2",
"kates-kitchen-san-francisco",
"triptych-san-francisco",
"le-petitts-kitchen-san-francisco-2",
"kyu3-noodle-and-bbq-san-francisco-4",
"ramen-yamadaya-san-francisco",
"jitlada-thai-cuisine-san-francisco",
"sakana-sushi-and-grill-san-francisco",
"picaro-san-francisco",
"smashburger-san-francisco-2",
"cocobang-san-francisco",
"golden-lotus-san-francisco",
"shido-san-francisco",
"sai-jai-thai-restaurant-san-francisco",
"grove-street-market-san-francisco",
"tanpopo-san-francisco",
"kim-thanh-restaurant-san-francisco",
"taqueria-el-buen-sabor-san-francisco",
"jaspers-corner-tap-and-kitchen-san-francisco-2",
"benders-bar-and-grill-san-francisco",
"the-boneyard-san-francisco",
"l-and-g-vietnamese-sandwich-san-francisco",
"firetrail-pizza-san-francisco",
"ha-nam-ninh-san-francisco-3",
"zen-yai-thai-restaurant-san-francisco",
"little-baobab-san-francisco",
"farallon-san-francisco",
"noir-lounge-san-francisco",
"panchitas-restaurant-no-2-san-francisco",
"double-play-bar-and-grill-san-francisco",
"metro-caffe-san-francisco",
"million-thai-restaurant-san-francisco",
"alis-pizzeria-san-francisco-3",
"kiss-seafood-san-francisco",
"the-morning-fix-san-francisco",
"bartlett-hall-san-francisco",
"kusina-ni-tess-san-francisco",
"coffee-bar-san-francisco",
"southpaw-bbq-and-southern-cooking-san-francisco",
"katana-ya-san-francisco",
"honey-honey-cafe-and-crepery-san-francisco",
"eiji-san-francisco",
"custom-burger-san-francisco",
"7-mission-restaurant-san-francisco-2",
"sliders-diner-san-francisco-2",
"makli-restaurant-san-francisco",
"fine-and-rare-san-francisco",
"pause-wine-bar-san-francisco",
"new-village-cafe-san-francisco",
"el-tonayense-taco-truck-san-francisco-8",
"gyro-king-san-francisco",
"estelas-fresh-sandwiches-san-francisco-2",
"indochine-vegan-san-francisco-2",
"the-one-sushi-and-lounge-san-francisco",
"the-one-sushi-and-lounge-san-francisco",
"zaoh-restaurant-san-francisco",
"the-kabob-job-san-francisco",
"ken-ken-ramen-san-francisco-2",
"h-cafe-san-francisco",
"carafe-cafe-san-francisco",
"golden-coffee-shop-san-francisco",
"yakiniq-bbq-san-francisco",
"stable-cafe-san-francisco",
"sushi-toni-san-francisco",
"victors-pizza-san-francisco",
"grilled-cheez-guy-san-francisco",
"taqueria-cancun-san-francisco-3",
"puccini-and-pinetti-san-francisco",
"limón-rotisserie-san-francisco-4",
"modern-thai-san-francisco",
"king-of-thai-noodle-house-san-francisco-3",
"opaque-dining-in-the-dark-san-francisco-2",
"dna-pizza-san-francisco",
"sing-sing-sandwich-shop-san-francisco",
"lers-ros-thai-san-francisco-3",
"the-vestry-san-francisco",
"flippers-a-gourmet-hamburger-place-san-francisco",
"takara-japanese-restaurant-san-francisco",
"caffetino-san-francisco",
"burgermeister-san-francisco",
"its-tops-coffee-shop-san-francisco",
"bdk-restaurant-san-francisco",
"two-jacks-seafood-san-francisco-3",
"kaka-udon-kitchen-san-francisco",
"cafe-taboo-san-francisco",
"carlins-cafe-san-francisco",
"evergreen-garden-restaurant-san-francisco",
"mission-chinese-food-san-francisco-4",
"shalala-ramen-san-francisco",
"el-tepa-taqueria-san-francisco",
"valencia-pizza-and-pasta-san-francisco",
"mangosteen-san-francisco",
"shabusen-restaurant-san-francisco-2",
"thai-idea-vegetarian-san-francisco-2",
"clares-deli-san-francisco-2",
"mums-home-of-shabu-shabu-san-francisco-2",
"hongry-kong-truck-san-francisco",
"arinell-pizza-san-francisco",
"the-mortimer-san-francisco",
"turk-and-larkin-deli-san-francisco",
"hayes-valley-bakeworks-san-francisco",
"ninas-cafe-san-francisco-2",
"klyde-cafe-and-wine-bar-san-francisco-5",
"cozy-cafe-san-francisco-2",
"cafe-du-nord-san-francisco-3",
"mifune-don-san-francisco",
"leatherneck-steakhouse-san-francisco-2",
"chicos-pizza-san-francisco",
"the-sage-cafe-san-francisco",
"estelas-fresh-sandwiches-san-francisco",
"paellas-ole-san-francisco",
"el-tesoro-taqueria-and-grill-san-francisco",
"thai-house-530-san-francisco",
"southern-pacific-brewing-san-francisco",
"southern-pacific-brewing-san-francisco",
"bissap-baobab-san-francisco-3",
"jasmins-cafe-san-francisco",
"biscuits-and-blues-san-francisco-4",
"after-osha-san-francisco-2",
"pork-store-cafe-san-francisco-2",
"the-orbit-room-san-francisco",
"the-market-san-francisco",
"the-phoenix-supper-club-san-francisco",
"frjtz-san-francisco-2",
"bhel-indian-food-popup-kitchen-san-francisco",
"morning-due-cafe-san-francisco",
"guerrero-market-and-deli-san-francisco",
"sears-fine-food-san-francisco",
"indigo-san-francisco",
"on-the-bridge-san-francisco",
"soluna-cafe-and-lounge-san-francisco",
"398-brasserie-san-francisco",
"all-star-cafe-san-francisco",
"scalas-bistro-san-francisco",
"little-sheep-mongolian-hot-pot-san-francisco-4",
"milan-pizza-san-francisco",
"dolores-park-cafe-san-francisco",
"henrys-hunan-san-francisco-4",
"miyabi-sushi-san-francisco-4",
"moz-cafe-san-francisco",
"city-chopsticks-san-francisco",
"brainwash-cafe-and-laundromat-san-francisco",
"mosto-san-francisco",
"el-toro-taqueria-san-francisco",
"greenlid-studiomix-san-francisco",
"r-caffe-san-francisco",
"whiz-burgers-drive-in-san-francisco",
"curry-up-now-san-francisco-5",
"i-luv-teriyaki-and-sushi-san-francisco",
"hans-coffee-shop-san-francisco",
"thanh-tam-ii-san-francisco",
"mustafios-pizza-san-francisco",
"playground-san-francisco",
"angel-cafe-and-deli-san-francisco",
"little-henrys-san-francisco",
"sparkys-pizza-and-diner-san-francisco",
"rheas-deli-and-market-san-francisco",
"the-grove-san-francisco",
"cable-car-cafe-san-francisco",
"kuletos-san-francisco-2",
"sushi-hana-san-francisco",
"new-korea-house-san-francisco",
"fresh-brew-coffee-san-francisco",
"lalitas-exotic-thai-cuisine-and-bar-san-francisco",
"the-little-chihuahua-san-francisco-4",
"deja-vu-pizza-and-pasta-san-francisco",
"taj-pizza-san-francisco-2",
"fresco-pizza-shawarma-san-francisco-4",
"samovar-tea-lounge-san-francisco",
"talbot-cafe-san-francisco",
"soma-inn-cafe-san-francisco",
"boba-delight-san-francisco-10",
"hayes-street-grill-san-francisco",
"market-street-grill-san-francisco",
"doobu-san-francisco",
"sushi-delight-san-francisco-2",
"pizza-zone-n-grill-san-francisco-3",
"mjs-cafe-san-francisco",
"dojima-ann-san-francisco",
"napoli-pizza-san-francisco",
"myconos-greek-san-francisco",
"paniolo-social-san-francisco",
"mr-nice-san-francisco",
"hoang-dat-coffee-shop-san-francisco",
"lanxang-kingdom-san-francisco",
"redpie-san-francisco",
"jones-san-francisco-3",
"nano-cafe-san-francisco",
"ithai-san-francisco",
"pinecrest-diner-san-francisco-2",
"linea-caffe-san-francisco",
"espetus-churrascaria-san-francisco",
"el-castillito-san-francisco",
"spicy-garden-san-francisco",
"osha-thai-restaurant-san-francisco-6",
"center-cafe-san-francisco",
"mini-cafe-san-francisco",
"lafayette-coffee-shop-san-francisco",
"bai-thong-thai-san-francisco",
"bistro-10un-san-francisco",
"brass-knuckle-san-francisco",
"pho-vietnam-san-francisco",
"mythic-pizza-san-francisco",
"allstar-cafe-san-francisco",
"raijin-sushi-san-francisco",
"el-sol-taqueria-san-francisco",
"azalinas-san-francisco-2",
"shalimar-resturant-san-francisco",
"regalito-rosticeria-san-francisco",
"liquid-culture-san-francisco",
"the-pastry-cupboard-san-francisco-3",
"grub-san-francisco-2",
"lunchbox-cafe-san-francisco",
"kens-kitchen-san-francisco",
"daily-grill-san-francisco-4",
"l-ottavo-ristorante-san-francisco",
"whitechapel-sf-san-francisco",
"isobune-sushi-san-francisco-3",
"bite-me-sandwiches-san-francisco-2",
"emos-cafe-san-francisco",
"farina-pizza-and-cucina-italiana-san-francisco",
"ali-babas-cave-san-francisco",
"yucatasia-san-francisco",
"cafe-mason-san-francisco",
"south-at-sfjazz-san-francisco",
"don-ramons-san-francisco-2",
"vincysunset-san-francisco",
"momi-tobys-revolution-cafe-and-art-bar-san-francisco",
"urban-tavern-san-francisco",
"annies-hot-dogs-and-pretzels-san-francisco",
"ramen-underground-san-francisco-4",
"crepevine-san-francisco-2",
"cafe-international-san-francisco",
"oui-chef-san-francisco",
"johnny-foleys-san-francisco",
"wes-burger-san-francisco",
"kippu-japanese-cuisine-san-francisco",
"munch-haven-san-francisco",
"bread-srsly-san-francisco",
"burger-house-san-francisco",
"machine-coffee-and-deli-san-francisco",
"felipes-steak-sundays-san-francisco",
"sutter-pub-san-francisco",
"lefty-o-douls-san-francisco-2",
"cha-ya-san-francisco-4",
"la-cafe-san-francisco",
"postrio-san-francisco",
"gourmet-carousel-san-francisco",
"pitas-cafe-san-francisco",
"jacobs-diner-san-francisco",
"casa-mexicana-san-francisco",
"el-salvador-san-francisco",
"punjab-kabab-house-san-francisco",
"hawker-fare-san-francisco",
"smitten-ice-cream-san-francisco-3",
"el-super-burrito-san-francisco",
"opera-plaza-deli-and-taqueria-san-francisco",
"cafe-asia-san-francisco",
"loris-diner-san-francisco-7",
"piccadilly-fish-and-chips-san-francisco",
"simply-delight-cafe-san-francisco",
"perfect-bite-san-francisco",
"king-ling-san-francisco",
"thai-stick-san-francisco-17",
"blur-san-francisco",
"kasa-indian-eatery-san-francisco-4",
"mifune-restaurant-san-francisco",
"saha-restaurant-san-francisco",
"kyoto-sushi-san-francisco",
"eat-curbside-san-francisco",
"loris-diner-san-francisco-6",
"sunflower-authentic-vietnamese-restaurant-san-francisco-2",
"coffee-mine-san-francisco",
"caffe-trinity-san-francisco-2",
"lumpia-cart-san-francisco",
"supreme-pizza-san-francisco",
"lees-sandwiches-san-francisco",
"etcetera-wine-bar-san-francisco",
"tads-steakhouse-san-francisco-2",
"colombini-italian-cafe-bistro-san-francisco",
"mels-drive-in-san-francisco-5",
"uncle-brothers-chicken-san-francisco",
"family-garden-san-francisco-3",
"bin-480-san-francisco",
"first-crush-restaurant-wine-bar-and-lounge-san-francisco-2",
"magic-wok-san-francisco",
"mixed-greens-san-francisco",
"mrcrave-bistro-café-san-francisco-6",
"el-cafetazo-san-francisco",
"cesarios-san-francisco",
"rite-spot-cafe-san-francisco",
"comida-yucatan-san-francisco",
"the-phoenix-bar-and-irish-gathering-house-san-francisco",
"fog-towne-cafe-san-francisco",
"izakaya-ninja-san-francisco",
"andersen-bakery-san-francisco",
"bar-norcini-san-francisco",
"izakaya-lounge-san-francisco",
"familia-lopez-cafe-san-francisco",
"burger-joint-san-francisco-2",
"l-emigrante-wine-bar-and-bistro-san-francisco",
"benihana-san-francisco",
"caffe-moda-san-francisco",
"village-pizzeria-san-francisco-5",
"the-old-siam-thai-restaurant-san-francisco",
"ah-lin-restaurant-san-francisco-2",
"café-at-the-opera-at-the-sfwm-san-francisco",
"piccolo-pizza-san-francisco-15",
"mikado-sushi-san-francisco-2",
"celtic-coffee-company-san-francisco",
"irving-pizza-san-francisco-5",
"muratas-cafe-hana-san-francisco",
"sight-glass-san-francisco",
"jade-cafe-san-francisco-2",
"level-iii-restaurant-san-francisco",
"fortress-solitude-and-san-francisco-pop-up-san-francisco",
"four-barrel-coffee-san-francisco",
"mélange-market-san-francisco-2",
"new-york-pizza-kitchen-san-francisco",
"bourbon-steak-san-francisco-3",
"the-whole-beast-san-francisco",
"taqueria-la-cumbre-san-francisco",
"slims-san-francisco-2",
"dux-san-francisco",
"mortons-the-steakhouse-san-francisco",
"mcdonalds-san-francisco-61",
"spiro-san-francisco",
"golden-gate-grill-san-francisco",
"cybelles-pizza-san-francisco-8",
"dong-baek-restaurant-san-francisco",
"sanppo-restaurant-san-francisco",
"bartlett-hall-san-francisco-2",
"cafe-encore-san-francisco",
"maxs-opera-cafe-san-francisco-2",
"samovar-tea-san-francisco-2",
"big-lantern-san-francisco",
"irmas-pampanga-restaurant-san-francisco",
"royalty-pizza-san-francisco",
"concordia-argonaut-san-francisco-2",
"puerto-alegre-san-francisco",
"jack-in-the-box-san-francisco",
"lalita-exotic-thai-cuisine-and-bar-san-francisco",
"pronto-pizza-san-francisco",
"thai-4-you-san-francisco",
"la-esquina-san-francisco",
"seoul-garden-san-francisco",
"ho-king-grill-san-francisco",
"bellissimo-pizza-san-francisco",
"cafe-la-taza-san-francisco",
"il-borgo-san-francisco",
"latte-express-san-francisco-4",
"daily-press-san-francisco",
"velvet-room-san-francisco",
"mint-cafe-san-francisco-4",
"original-perfect-hamburger-san-francisco",
"cassia-san-francisco",
"tu-kim-coffee-shop-san-francisco",
"marrakech-moroccan-restaurant-san-francisco",
"mayes-oyster-house-san-francisco",
"estrella-taqueria-san-francisco",
"project-juice-san-francisco-6",
"focaccia-market-and-bakery-san-francisco-6",
"wan-kee-restaurant-san-francisco-2",
"nizarios-pizza-san-francisco-5",
"emperors-kitchen-san-francisco",
"lt-waffle-san-francisco",
"chipotle-mexican-grill-san-francisco-4",
"muka-san-francisco-2",
"roh-bar-and-restaurant-san-francisco",
"la-estrellita-snack-san-francisco",
"los-compadres-taco-truck-san-francisco",
"stanza-coffee-san-francisco",
"el-pípila-san-francisco-2",
"haight-pizza-san-francisco",
"maru-sushi-san-francisco",
"cafe-bellini-san-francisco",
"market-mayflower-and-deli-san-francisco",
"san-wang-restaurant-san-francisco",
"cathay-express-restaurant-san-francisco",
"joliba-cafe-san-francisco-4",
"lets-jam-cafe-san-francisco",
"taqueria-la-paz-san-francisco-2",
"all-star-tamales-san-francisco-2",
"cafe-bellatrix-san-francisco",
"la-noísette-cafe-and-delí-san-francisco",
"el-rincón-yucateco-san-francisco-3",
"blondies-pizza-san-francisco",
"l-and-m-cafe-san-francisco",
"cafe-bean-san-francisco",
"cafe-majestic-san-francisco-2",
"tacos-el-tuca-san-francisco",
"fuel-lounge-san-francisco-san-francisco",
"borobudur-restaurant-san-francisco",
"jeanne-d-arc-san-francisco",
"yo-yo-restaurant-san-francisco",
"dolce-san-francisco-2",
"caffe-delle-stelle-san-francisco",
"taqueria-vallarta-san-francisco-4",
"h-and-w-liquors-san-francisco",
"roxanne-cafe-san-francisco",
"cable-car-city-pub-and-cafe-san-francisco-2",
"ben-thai-cafe-san-francisco",
"don-agapitos-san-francisco",
"mz-liz-san-francisco",
"q-mart-san-francisco",
"little-cafe-san-francisco",
"lets-be-frank-san-francisco-6",
"pier-45-san-francisco",
"costco-food-court-san-francisco",
"hais-restaurant-san-francisco",
"volare-pizza-and-restaurant-san-francisco",
"hayes-market-san-francisco",
"busters-cheesesteaks-san-francisco",
"akikos-sushi-bar-san-francisco",
"maxfields-house-of-caffeine-san-francisco",
"golden-kim-tar-san-francisco",
"diana-market-2-san-francisco",
"para-bellum-verde-san-francisco",
"el-tonayense-taco-truck-san-francisco-4",
"we-be-sushi-san-francisco-2",
"taco-bell-san-francisco-9",
"cafe-kuleto-san-francisco",
"mason-street-deli-san-francisco",
"hotel-whitcomb-restaurant-san-francisco",
"cityscape-bar-and-restaurant-san-francisco-2",
"las-cocineras-restaurant-san-francisco",
"tacos-santana-taco-truck-san-francisco",
"olympic-flame-cafe-san-francisco",
"go-getters-pizza-san-francisco-3",
"pizza-by-the-slice-san-francisco",
"taqueria-las-palmas-san-francisco",
"shanghai-china-restaurant-san-francisco",
"bar-1915-san-francisco",
"krua-thai-san-francisco",
"them-ky-san-francisco",
"new-star-restaurant-san-francisco",
"curiosity-island-san-francisco",
"street-fixe-san-francisco",
"lacys-malaysian-crepes-san-francisco",
"oriental-restaurant-san-francisco",
"momos-san-francisco-4",
"h2o-cafe-san-francisco",
"cable-55-san-francisco",
"alhamra-san-francisco-4",
"we-sushi-san-francisco",
"red-rooster-tacos-san-francisco",
"jebena-cafe-san-francisco-2",
"sf-pizza-san-francisco",
"bayou-by-the-bay-san-francisco",
"graze-salad-bar-san-francisco",
"noahs-bagels-san-francisco-4",
"alborz-persian-cuisine-san-francisco",
"chicos-pizza-san-francisco-4",
"taco-window-san-francisco",
"taco-window-san-francisco",
"casbah-halal-market-san-francisco",
"bang-san-thai-san-francisco",
"milan-pizza-san-francisco-2",
"blue-cube-san-francisco",
"quickly-san-francisco-16",
"mint-cafe-san-francisco-5",
"thai-stick-san-francisco-16",
"creperie-saint-germain-san-francisco-3",
"carls-jr-san-francisco-3",
"kanom-thai-san-fran-san-francisco",
"wing-lum-san-francisco",
"bamboo-restaurant-san-francisco-2",
"castillito-yucateco-san-francisco-5",
"carls-jr-san-francisco-2",
"red-jade-san-francisco",
"happy-donuts-san-francisco-17",
"brooklyn-pizza-san-francisco",
"mochas-cafe-and-grill-san-francisco",
"davids-delicatessen-san-francisco",
"safeway-san-francisco-18",
"cafechanta-raska-san-francisco",
"anzu-san-francisco",
"kyoto-noodle-express-san-francisco",
"cafe-bellini-san-francisco-5",
"pierogisf-san-francisco",
"thep-phanom-thai-restaurant-san-francisco-2",
"sonias-san-francisco",
"annies-hot-dog-cart-san-francisco-4",
"crimson-lounge-san-francisco",
"taste-by-four-seasons-san-francisco",
"rotana-pizza-san-francisco",
"pesbas-fish-and-chicken-san-francisco",
"papa-johns-pizza-san-francisco-2",
"ultimate-souvlaki-san-francisco-2",
"gracies-cafe-san-francisco",
"caffe-mio-san-francisco",
"thai-cooking-san-francisco",
"little-red-riding-truck-san-francisco",
"lost-ladle-cafe-san-francisco",
"s-and-s-caesars-cafe-san-francisco",
"pita-pita-san-francisco",
"subway-san-francisco-11",
"hot-dog-stand-san-francisco-4",
"pub-city-cafe-san-francisco",
"zingari-ristorante-san-francisco",
"lung-shan-restaurant-san-francisco-2",
"sushi-boat-restaurant-san-francisco",
"tamale-man-san-francisco",
"tikka-masala-san-francisco-3",
"city-hall-cafe-san-francisco",
"kfc-san-francisco-7",
"tower-cafe-san-francisco-2",
"j-and-e-delicatessen-san-francisco",
"piraat-pizzeria-and-rotisserie-san-francisco-2",
"cafe-450-eurest-dining-san-francisco",
"duc-loi-supermarket-san-francisco",
"california-culinary-academy-san-francisco-3",
"bella-lucca-san-francisco",
"the-oak-room-restaurant-san-francisco",
"the-oak-room-restaurant-san-francisco",
"mission-hunan-restaurant-san-francisco",
"supremo-pizza-san-francisco",
"doña-marta-san-francisco-2",
"crepes-escape-san-francisco",
"adams-grub-truck-san-francisco-2",
"drip-san-francisco",
"green-salads-san-francisco",
"allstar-donuts-and-sandwich-san-francisco",
"adobo-roll-o-san-francisco",
"tian-sing-san-francisco",
"round-table-pizza-san-francisco-7",
"icu-market-deli-and-ice-cream-san-francisco",
"arabian-sky-san-francisco",
"jins-cafe-san-francisco-3",
"delish-deli-san-francisco",
"pyongyang-express-san-francisco",
"mels-diner-san-francisco-51",
"al-fanous-san-francisco",
"le-colonial-san-francisco",
"sukhis-gourmet-indian-foods-san-francisco-3",
"sunflower-restaurant-san-francisco",
"allstar-donuts-and-burgers-san-francisco",
"bay-city-pizza-san-francisco",
"mobowl-san-francisco",
"the-chapel-san-francisco",
"tres-truck-san-francisco-4",
"burger-king-san-francisco-9",
"veracios-pizza-san-francisco",
"pizza-prada-san-francisco",
"mi-tierra-san-francisco",
"e-and-m-cafe-san-francisco",
"alfaro-catering-san-francisco-2",
"american-cafe-san-francisco",
"ramen-yokocho-fest-2014-san-francisco",
"east-of-eden-san-francisco",
"boston-market-san-francisco",
"horobitsu-no-ie-san-francisco",
"infusion-lounge-san-francisco",
"tapeo-sf-san-francisco",
"itea-house-san-francisco",
"pakwan-restaurant-o-farrell-street-san-francisco",
"indian-oven-san-francisco-2",
"joshs-diner-san-francisco-2",
"darbar-restaurant-san-francisco",
"50-on-eight-restaurant-san-francisco",
"martin-and-hunter-foods-san-francisco",
"togos-sandwiches-san-francisco-2",
"submarine-sandwiches-san-francisco",
"ox-cafe-and-tea-san-francisco",
"vietnam-house-san-francisco",
"the-pour-house-san-francisco",
"buckhorn-grill-san-francisco-5",
"new-lalita-san-francisco",
"pho-kim-long-san-francisco",
"bacon-dog-cart-san-francisco",
"can-tho-coffee-san-francisco",
"carmelinas-cafe-san-francisco",
"sunflower-cafe-san-francisco",
"cafe-duong-dong-san-francisco",
"cafe-duong-dong-san-francisco",
"mcdonalds-san-francisco-31",
"dobbs-bar-san-francisco-2",
"tins-cafe-san-francisco",
"sidewalk-cafe-san-francisco-2",
"hot-dog-alley-san-francisco-3",
"broth-baby-san-francisco",
"bien-hoa-san-francisco",
"om-restaurant-san-francisco",
"jennys-restaurant-san-francisco",
"kushi-tsuru-san-francisco",
"ooh-la-la-lumpia-san-francisco-2",
"docs-classic-oakland",
"this-is-it-glatt-kosher-restaurant-san-francisco",
"chez-boudin-san-francisco",
"king-of-panda-san-francisco",
"sunny-cafe-san-francisco",
"the-sultan-san-francisco",
"hong-kong-express-cafe-san-francisco",
"first-restaurant-san-francisco",
"pizza-hut-san-francisco-3",
"alfaro-catering-san-francisco",
"isobune-san-francisco",
"kfc-san-francisco-24",
"new-home-menu-restaurant-san-francisco",
"golden-gate-tap-room-san-francisco",
"san-francisco-main-library-cafe-san-francisco",
"farina-focaccia-and-cucina-italiana-san-francisco",
"los-poblanos-san-francisco",
"san-francisco-pizza-company-san-francisco",
"fiesta-tacos-san-francisco",
"subway-san-francisco-31",
"subway-san-francisco-8",
"bacio-restaurant-san-francisco",
"calpe-paella-san-francisco",
"crawfish-fusion-san-francisco",
"subway-san-francisco-27",
"rotana-shawarma-san-francisco",
"valencia-and-kebab-san-francisco",
"subway-san-francisco",
"subway-san-francisco-47",
"subway-san-francisco-37",
"wallys-fried-chicken-san-francisco",
"charleys-grilled-subs-san-francisco",
"tacomyface-san-francisco",
"sfq-original-sf-bbq-sauce-san-francisco",
"el-tin-tan-san-francisco",
"subway-san-francisco-32",
"v-and-n-express-chinese-food-san-francisco",
"ricas-pupusas-restaurant-san-francisco",
"pompodoris-pizzas-san-francisco",
"info-cafe-san-francisco",
"curry-in-hurry-san-francisco",
"the-taco-bike-san-francisco",
"burger-king-san-francisco",
"san-buena-taco-truck-san-francisco-4",
"burger-king-san-francisco-4",
"k-pow-san-francisco",
"burger-king-san-francisco-8",
"osakaya-restaurant-san-francisco",
"pagoda-cafe-san-francisco",
"persimmon-san-francisco-2",
"eatwithlaila-san-francisco",
"bombay-indian-restaurant-san-francisco",
"new-minnys-san-francisco",
"signor-pizza-san-francisco",
"heaven-no-title",
"asia-express-san-francisco",
"burger-king-san-francisco-12",
"pakkain-macro-marketing-as-goodies-san-francisco-2",
"southern-slant-san-francisco",
"cafe-fresco-san-francisco",
"star-wok-express-san-francisco",
"bamboo-pizza-san-francisco",
"pizza-town-san-francisco",
"sweet-creme-brulee-san-francisco-2",
"castillo-yucateco-san-francisco",
"mortons-bar-12-21-san-francisco",
"dominos-pizza-san-francisco-5",
"clock-bar-san-francisco",
"wellcome-bar-and-restaurant-san-francisco",
"subway-san-francisco-73",
"girl-friday-zeppole-san-francisco",
"new-delhi-restaurant-san-francisco",
"kim-wong-keltner-san-francisco",
"chili-dogs-san-francisco",
"new-land-cafe-san-francisco",
"daffodil-san-francisco",
"ys-gourmet-productions-san-francisco",
"post-street-hair-and-nail-care-san-francisco",
"tl-cafe-and-laundromat-san-francisco",
"mochas-cafe-and-grill-san-francisco-3",
"la-canasta-san-francisco-2",
"clara-gil-stephens-in-house-café-san-francisco",
"twin-cafe-san-francisco",
"sakana-san-francisco",
"kit-fox-san-francisco",
"kim-hoang-coffee-and-fast-food-san-francisco",
"crab-shack-san-francisco",
"sky-high-cafe-san-francisco",
"sky-high-cafe-san-francisco",
"one-world-cafe-san-francisco-2",
"orlandos-san-francisco",
"art-of-the-falafel-san-francisco",
"bistro-burger-san-francisco-5",
"archimedes-kitchen-san-francisco",
"native-sons-bbq-san-francisco",
"taco-bell-san-francisco-12",
"uma-casa-san-francisco",
"whole-beast-seafood-bar-san-francisco",
"chicago-style-hot-dogs-san-francisco",
"sushi-friend-san-francisco",
"cable-car-pizza-san-francisco-6",
"tamales-la-oaxaquena-san-francisco",
"aids-health-alliance-cafe-san-francisco",
"yemeni-kitchen-san-francisco",
"youki-ramen-san-francisco",
"mahoneys-seafood-san-francisco",
"right-way-market-and-deli-san-francisco"]

In [ ]:
# Imports
import string
from lxml import html
import requests

# Yelp unique url endings for each restaurant
RESTAURANTS = bigbizlist

start_urls = [ 'http://www.yelp.com/biz/%s' % s for s in RESTAURANTS]
num_reviews = 2000 # Number of reviews you want
page_order = range(0, (num_reviews+1), 40)
big_review_dict = {}

for ur in start_urls:
    for o in page_order:
        page = requests.get(ur + ("?start=%s" % o))
        tree = html.fromstring(page.text)

        # This will extract the name of the business
        title = tree.xpath('//h1[@itemprop="name"]/text()')
        title = title[0].strip()

        # This will create a list of reviews
        reviews = tree.xpath('//p[@itemprop="description"]/text()')
        mod_reviews = []
        for rev in reviews:
            mod_rev = filter(lambda x: x in string.printable, rev)
            mod_reviews.append(mod_rev)


        # This will add the reviews to a dictionary
        if title in big_review_dict:
            big_review_dict[title] += mod_reviews
        else:
            big_review_dict[title] = mod_reviews